In [357]:
import pandas as pd
from config import config
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
    
plt.rcParams['figure.figsize'] = [10, 8]

In [358]:
# create cache directory
os.makedirs('cache', exist_ok=True)

In [359]:
from pandas import DataFrame

def meta(c:dict) -> dict:
    df = pd.read_csv(c['data_path'], dtype=str)
    
    c['size'] = len(df)
    c['columns'] = list(df.columns)
    return c


In [360]:
px.bar?

Object `px.bar` not found.


assuming you have data and its placed where is should ("python3 fetch_data.py")

Business Understanding

* Is the salary correlated with some programming language, framework or platform?
* How much influence have the country the developer is working ?
* Are we able to predict salary based on the stack overflow data?

In [361]:
config[-1]

{'year': '2020',
 'url': 'https://drive.google.com/uc?id=1dfGerWeWkcyQ9GX9x20rdSGj7WtEpzBB&export=download',
 'local_path': 'data/2020.zip',
 'unpack_path': 'data/unpack/2020.zip',
 'data_path': 'data/unpack/2020.zip/survey_results_public.csv'}

In [362]:
df_raw = pd.read_csv(config[-1]['data_path'], dtype=str)

salary encoded here as `ConvertedComp` - its in USD

In [363]:
df_raw.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,EUR,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",White or of European descent,Man,"Languages, frameworks, and other technologies ...",Slightly satisfied,I am not interested in new job opportunities,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,.NET Core;Xamarin,.NET;.NET Core,Microsoft Teams;Microsoft Azure;Trello,Confluence;Jira;Slack;Microsoft Azure;Trello,No,Somewhat important,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,Often: 1-2 days per week or more,Start a free trial;Ask developers I know/work ...,Amused,Stack Overflow (public Q&A for anyone who codes),Visit Stack Overflow;Go for a walk or other ph...,Windows,2 to 9 employees,Android;iOS;Kubernetes;Microsoft Azure;Windows,Windows,NaN,Straight / Heterosexual,No,"No, not at all",NaN,Multiple times per day,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,GBP,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,NaN,Very dissatisfied,I am not interested in new job opportunities,Python;Swift,JavaScript;Swift,React Native;TensorFlow;Unity 3D,React Native,Github;Slack,Confluence;Jira;Github;Gitlab;Slack,NaN,NaN,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,NaN,NaN,Amused,Stack Overflow (public Q&A for anyone who code...,Visit Stack Overflow;Go for a walk or other ph...,MacOS,"1,000 to 4,999 employees",iOS;Kubernetes;Linux;MacOS,iOS,I have little or no influence,NaN,Yes,"Yes, definitely",Less than once per month or monthly,Multiple times per day,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Once a decade,NaN,NaN,No,NaN,NaN,NaN,Stack Overflow (public Q&A for anyone who codes),NaN,Linux-based,NaN,NaN,NaN,NaN,NaN,Yes,"Yes, somewhat",A few times per month or weekly,Daily or almost daily,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25,18,NaN,NaN,NaN,Albania,Albanian lek,ALL,NaN,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,White or of European descent,Man,Flex time or a flexible schedule;Office enviro...,Slightly dissatisfied,"I’m not actively looking, but I am open to new...",NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,Not at all important/not necessary,Curious about other opportunities;Wanting to w...,NaN,Once a year,Not sure,Yes,Yes,Occasionally: 1-2 days per quarter but less th...,NaN,NaN,Stack Overflow (public Q&A for anyone who code...,NaN,Linux-b

In [364]:
len(df_raw)

64461

In [365]:
df_raw.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'Age', 'Age1stCode', 'CompFreq',
       'CompTotal', 'ConvertedComp', 'Country', 'CurrencyDesc',
       'CurrencySymbol', 'DatabaseDesireNextYear', 'DatabaseWorkedWith',
       'DevType', 'EdLevel', 'Employment', 'Ethnicity', 'Gender', 'JobFactors',
       'JobSat', 'JobSeek', 'LanguageDesireNextYear', 'LanguageWorkedWith',
       'MiscTechDesireNextYear', 'MiscTechWorkedWith',
       'NEWCollabToolsDesireNextYear', 'NEWCollabToolsWorkedWith', 'NEWDevOps',
       'NEWDevOpsImpt', 'NEWEdImpt', 'NEWJobHunt', 'NEWJobHuntResearch',
       'NEWLearn', 'NEWOffTopic', 'NEWOnboardGood', 'NEWOtherComms',
       'NEWOvertime', 'NEWPurchaseResearch', 'NEWPurpleLink', 'NEWSOSites',
       'NEWStuck', 'OpSys', 'OrgSize', 'PlatformDesireNextYear',
       'PlatformWorkedWith', 'PurchaseWhat', 'Sexuality', 'SOAccount',
       'SOComm', 'SOPartFreq', 'SOVisitFreq', 'SurveyEase', 'SurveyLength',
       'Trans', 'UndergradMajor', 'WebframeDesireNextYear',
  

The data of stackoverflow contains a lot of columns (61 in total). Most of the data is a string of a set of possible answers. Only a small amount contains floating point values. In total, we have 64461 answers, where about 53.9% (34756) contain an answer related to their current job salary, less than we got compared to the job satisfaction we analysed at the beginning of this year (70%).

In [366]:
df = df_raw[~ df_raw['ConvertedComp'].isnull()]

In [367]:
float(len(df)) / len(df_raw)

0.5391787282232667

In [368]:
# del df_raw

In [369]:
len(df)

34756

In [370]:
df['CompTotal'].describe()

count     34756
unique     2997
top       1e+05
freq        767
Name: CompTotal, dtype: object

In [371]:
set(df['CompFreq'])

{'Monthly', 'Weekly', 'Yearly'}

We have a total 2997 unique values. But this values can be references as 'Monthly', 'Weekly' and 'Yearly' - that needs to be considered then doing the data preparation.

# Section 3: Data Preparation

## Selection

All rows that don't have a salary defined, will where removed from the dataset.

# Construct

I created new columns out of the existings ones in order to extract the features we want to analyse. This was done for all columns, resulting in a very wide table. Run `preprocessing.py` in the project directory.

In [372]:
# Stolen from myself: https://github.com/dariusgm/stackoverflow2020/blob/main/Blog.ipynb

def to_list(elements) -> list:
    '''
    Return a list of elements that where separated by ";"

    :param elements: the (potencial) elements to convert to a list, can be (str) or (np.nan)
    :returns: (list[str]) list of elements with 0 or more elements
    
    Example:
    >>> import numpy as np; to_list(np.nan)
    []
    
    >>> to_list("c++;c#")
    ["c++", "c#"]
    '''

    if type(elements) != str: # assuming nan
        return []
    else:
        return elements.split(";")
    


def df_to_features(df: pd.DataFrame, column: str) -> list:
    '''
    Explode a column of a pd.DataFrame containing several features
    
    :param df: (pd.DataFrame) DataFrame that have the column to explode
    :param column: (str) The name of the column with the values to explode
    
    :returns: (list[dict]) A list of dicts that have a true value for a particular line
    
    Example:
    >>> df_to_features(pd.DataFrame([{'a': 'b;c'}]), 'a')
    [{'a_b': 1, 'a_c': 1}]
    '''
    
    records = []
    for index, row in df.iterrows():
        line = {}
        elements = to_list(row[column])
        if len(elements)>0:
            for e in elements:
                # comparing to original code, I add here the original column name,
                # preventing collisions with Yes / No Answers
                line[f"{column}_{e}"] = 1
                    
        records.append(line)
        
    return records

def df_extract_features(df: pd.DataFrame, column: str) -> pd.DataFrame:
    '''
    Extract features of a particular column and returns the filled pd.DataFrame back
    
    :param df: (pd.DataFrame) The DataFrame with the data to be extracted
    :param column: (str) The column with the data to extract
    :returns: pd.DataFrame with feature columns, filled missing values with 0 
    
    Example:
    >>> df_extract_features(pd.DataFrame([{'a': 'b;c'}, {'a': 'c'}]), 'a')
    pd.DataFrame([{'b': 1.0, 'c': 1}, {'b': 0.0, 'c': 1}])
    '''
    return pd.DataFrame(df_to_features(df, column)).fillna(0.)


def calculate_and_save(df: DataFrame, column:str,  year:int) -> pd.DataFrame:
    '''
    Calculate features and cache result. Further calls will return the precalculated results
    
    :param df: (pd.DataFrame) The DataFrame with the data to be processed
    :param column: (str) The column with the data to process
    :param year: (int) the year of the data, used as cache key
    :returns: go.Figure, ready to use plotly figure
    '''
    
    cache_key = f"{year}_{column}.csv"
    cache_path = os.path.join('cache', cache_key)
    if os.path.exists(cache_path):
        return pd.read_csv(cache_path).set_index('id')
    df = df_extract_features(df, column)
    df.to_csv(cache_path, index_label='id')
    return df

def plot_distribution(df: DataFrame, column:str, year:int, sort_by:str='values') -> go.Figure:
    '''
    Plots the sum of the positives of a partical column with multilable values.
    
    :param df: (pd.DataFrame) The DataFrame with the data to be plotted
    :param column: (str) The column with the data to plot
    :returns: go.Figure, ready to use plotly figure
    '''
    
    df_cached = calculate_and_save(df, column, year)
    df_sum = df_cached.sum().to_frame()
    if sort_by == 'values':
        df_sorted = df_sum.sort_values(by=0, ascending=True)
    elif sort_by == 'index': 
        df_sorted = df_sum.sort_index(ascending=True)
    
    y = list(map(lambda x: x.split('_')[-1], df_sorted.index))
    x = df_sorted.values.flatten()
    
    fig = go.Figure(data=[go.Bar(x=x, y=y, text=y, orientation='h')])
    return fig


In [373]:
plot_distribution(df, 'MainBranch', 2020).show()

In [374]:
plot_distribution(df, 'Hobbyist', 2020).show()


In [375]:
plot_distribution(df, 'Age', 2020, 'index').show()

In [376]:
plot_distribution(df, 'Age1stCode', 2020, 'index').show()

age can be converted into ranges of age, as 

In [377]:
plot_distribution(df, 'CompFreq', 2020).show()

19107 yearly salary information, 14680 Monthly and 969 Weekly 

In [378]:
plot_distribution(df, 'Country', 2020, 'index').show()

In [379]:
plot_distribution(df, 'DatabaseDesireNextYear', 2020).show()

In [380]:
plot_distribution(df, 'DatabaseWorkedWith', 2020).show()

In [381]:
plot_distribution(df, 'EdLevel', 2020).show()

In [382]:
plot_distribution(df, 'Employment', 2020).show()

In [383]:
plot_distribution(df, 'Ethnicity', 2020).show()

In [384]:
plot_distribution(df, 'Gender', 2020).show()

In [385]:
plot_distribution(df, 'JobFactors', 2020).show()

In [386]:
plot_distribution(df, 'JobSat', 2020).show()

In [387]:
plot_distribution(df,  'JobSeek', 2020).show()

In [388]:
plot_distribution(df, 'LanguageDesireNextYear', 2020).show()

In [389]:
plot_distribution(df, 'LanguageWorkedWith', 2020).show()

In [390]:
plot_distribution(df, 'MiscTechDesireNextYear', 2020).show()

In [391]:
plot_distribution(df, 'MiscTechWorkedWith', 2020).show()

In [392]:
plot_distribution(df, 'NEWCollabToolsDesireNextYear', 2020).show()

In [393]:
plot_distribution(df, 'NEWCollabToolsWorkedWith', 2020).show()

In [394]:
plot_distribution(df, 'NEWDevOps', 2020).show()

In [395]:
plot_distribution(df,  'NEWDevOpsImpt', 2020).show()

In [396]:
plot_distribution(df, 'NEWEdImpt', 2020).show()

In [397]:
plot_distribution(df, 'NEWJobHunt', 2020).show()

In [398]:
plot_distribution(df, 'NEWJobHuntResearch', 2020).show()

In [399]:
plot_distribution(df, 'NEWLearn', 2020).show()

In [400]:
plot_distribution(df, 'NEWOffTopic', 2020).show()

In [401]:
plot_distribution(df, 'NEWOnboardGood', 2020).show()

In [402]:
plot_distribution(df, 'NEWOtherComms', 2020).show()

In [403]:
plot_distribution(df, 'NEWOvertime', 2020).show()

In [404]:
plot_distribution(df, 'NEWPurchaseResearch', 2020).show()

In [405]:
plot_distribution(df, 'NEWPurpleLink', 2020).show()

In [406]:
plot_distribution(df, 'NEWSOSites', 2020).show()

In [407]:
plot_distribution(df, 'NEWStuck', 2020).show()

In [408]:
plot_distribution(df, 'OpSys', 2020).show()

In [409]:
plot_distribution(df, 'OrgSize', 2020).show()

In [410]:
plot_distribution(df, 'PlatformDesireNextYear', 2020).show()

In [411]:
plot_distribution(df, 'PlatformWorkedWith', 2020).show()

In [412]:
plot_distribution(df, 'PurchaseWhat', 2020).show()

In [413]:
plot_distribution(df, 'Sexuality', 2020).show()

In [414]:
plot_distribution(df, 'SOAccount', 2020).show()

In [415]:
plot_distribution(df, 'SOComm', 2020).show()

In [416]:
plot_distribution(df, 'SOPartFreq', 2020).show()

In [417]:
plot_distribution(df, 'SOVisitFreq', 2020).show()

In [418]:
plot_distribution(df, 'SurveyEase', 2020).show()

In [419]:
plot_distribution(df, 'SurveyLength', 2020).show()

In [420]:
plot_distribution(df, 'Trans', 2020).show()

In [421]:
plot_distribution(df, 'UndergradMajor', 2020).show()

In [422]:
plot_distribution(df, 'WebframeDesireNextYear', 2020).show()

In [423]:
plot_distribution(df, 'WebframeWorkedWith', 2020).show()

In [424]:
plot_distribution(df, 'WelcomeChange', 2020).show()

In [429]:
plot_distribution(df, 'WorkWeekHrs', 2020).show()

In [430]:
plot_distribution(df, 'YearsCode', 2020, 'index').show()

In [427]:
plot_distribution(df, 'YearsCodePro', 2020).show()

* Age, Age1stCode, YearsCode, YearsCodePro for similar ranges
* WorkWeekHrs for similar ranges
* Country for regions